# Text parser for volume two

In [45]:
# DO NOT USE

import os
with open("./input/raw.txt", "r") as infile:
    data = infile.readlines()
    
    # Usable Parts of Section Two
    with open("input/Southern.txt", 'w') as outfile:
        outfile.writelines(data[3074:18607])
        
    # Usable Parts of Section Three
    with open("Northern.txt", 'w') as outfile:
        outfile.writelines(data[18611:39874])
        
    # Usable Parts of Section Four
    with open("Jiuliyang.txt", 'w') as outfile:
        outfile.writelines(data[39727:43643])

In [32]:
# DO NOT USE

import re


def refine(file):
    # Function to process raw data for future further operation
    
    buffer = None
    
    with open(file, "r+") as f:
        buffer = f.read()
        f.seek(0)
        
        # Delete Watermarks
        buffer = re.sub("Kenneth Dean and Zhenman ZHENG - 978-90-47-44017-8", "", buffer).rstrip()
        buffer = re.sub("Downloaded from Brill.com02/18/2020 04:31:15AM", "", buffer).rstrip()
        buffer = re.sub("via National University of Singapore - YNC Library", "", buffer).rstrip()
        
        # Delete Headers
        buffer = re.sub("the southern irrigated plain 南洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the northern irrigated plain 北洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the jiuliyang irrigated plain 九里洋水利系统", "", buffer).rstrip()
        buffer = re.sub("part (one|two|three|four|five)", "", buffer).rstrip()
        
        # Delete Page Numbers
        buffer = re.sub("\n[0-9]+\n", "", buffer).rstrip()
        buffer = re.sub("\f(.*?)\n", "", buffer).rstrip()
        buffer = re.sub("\n\n(A|V)[0-9]+\n\n\n", "", buffer).rstrip()
        
        # Delete blank lines
        buffer = re.sub("\n+", " ", buffer).rstrip()
        
        # Add line breaks at each new entry
        buffer = re.sub("((A|V)[0-9]{3})", "\n\g<1>", buffer)

        f.truncate()        
        f.write(buffer)

In [46]:
# DO NOT USE

refine("Southern.txt")
refine("Northern.txt")
refine("Jiuliyang.txt")



In [1]:
# Read god names and create JSON of them
import os, json

godsList= []

with open("input/list_of_gods.txt", 'r') as infile:
    data = infile.readlines()
    
    for line in data:
        EngText=" ".join(line.split()[:-1]).split("/")
        
        lineEntry = {
            "Name": EngText[0],
            "Name_zh": line.split()[-1]
        }
        
        if len(EngText) > 1:
            lineEntry["Aliases"] = EngText[1:]
        else:
            lineEntry["Aliases"] = []
        
        godsList.append(lineEntry)
        
    infile.close()

with open("./output/god_names.json", "w") as outfile:
                json.dump(godsList, outfile, ensure_ascii=False)
                outfile.close()

In [2]:
# Load village mapping functions
import re, os, json

village_index = []

with open("./input/contents.txt") as infile:
    for line in infile:
        buffer = re.split("(V[0-9]{3})", line)      
        entry = {
            "UID_A": buffer[0][0:4],
            "Name": " ".join(buffer[0].split(" ")[1:-2]),
            "Name_zh": buffer[0].split(" ")[-2]
        }

        runs = int((len(buffer)-1)/2 + 1)
        villageList=[]

        for i in range(1,runs):
            village_entry = {
                "UID_V": buffer[i*2-1],
                "Name": buffer[i*2].split(" ")[1],
                "Name_zh": buffer[i*2].split(" ")[2]
            }
            villageList.append(village_entry)

        entry["Num_Villages"] = len(villageList)
        entry["Villages"] = villageList

        village_index.append(entry)

with open("./output/village_index.json", "w") as outfile:
    json.dump(village_index, outfile, ensure_ascii=False)
    outfile.close()

infile.close()
    
#search function
def whatAlliance(searchterm):
    for alliance in village_index:
        for village in alliance["Villages"]:
            if village["UID_V"] == searchterm:
                return alliance
        
    return -1
    

In [3]:
# Load surname mapping functions

with open('./input/Surname_Groups.json', "r") as surnameFile:
    surnameGroups = json.load(surnameFile)
surnameFile.close()

#search function
def whatSurnames(searchterm):
    for village in surnameGroups:
        if village["UID"] == searchterm:
            return village
    return -1


In [14]:
# Load Text and print them into seperate text files.
import json

def createVillageFiles(path, irrNetwork):
    
    with open(path) as infile:
        for line in infile:
            if line[0] == "V":
                uid = line[0:4]                
                buffer1 = line.split(" ", 3)
                buffer2 = buffer1[3].partition("2. Surname groups")
                buffer3 = buffer2[2].partition("3. Village temples")
                buffer4 = buffer3[2].partition("4. Rituals")
                buffer5 = buffer4[2].partition("Birthday celebrations of the gods")
            
                # Search for Alliance details
                allianceDetails = whatAlliance(uid)
            
            
                entry = {
                    "UID": buffer1[0],
                    "Name": buffer1[1],
                    "Name_zh": buffer1[2],
                    "Alliance": allianceDetails["Name"],
                    "Alliance_UID": allianceDetails["UID_A"],
                    "Alliance_zh": allianceDetails["Name_zh"],
                    "Irrigation_Region": irrNetwork,
                    "Raw_Text": {                        
                        "Village_Settlement": buffer2[0][22:],
                        "Surname_Groups": buffer3[0][1:],
                        "Village_Temples": buffer4[0][1:],
                        "Rituals": {
                            "Yuanxiao processions": buffer5[0][23:],
                            "Birthday celebrations of the gods": buffer5[2][1:].strip()
                        }
                    }
                }
                
                # Add administrative village if applicable
                admin_village = re.search("\w* Administrative Village\s([^\.\s]+)", entry["Raw_Text"]["Village_Settlement"])
                if admin_village != None:
                    split_words = admin_village.group(0).split(" ")
                    entry["Raw_Text"]["Administrative_Village"] = " ".join(split_words[:-1])
                    entry["Raw_Text"]["Administrative_Village_zh"] = "".join(split_words[-1])                    
                else:
                    entry["Administrative_Village"] = "-1"
                    entry["Administrative_Village_zh"] = "-1"
                    
               # Find Population
                pop = re.search("[0-9,]*(?=\speople|\sresidents|\sinhabitants)", entry["Raw_Text"]["Village_Settlement"])
                
                if pop == None:
                    pop = re.search("[\w\s]*(people|residents|inhabitants|population)[\w\s]*", entry["Raw_Text"]["Village_Settlement"])
                    
                    if pop == None:
                        entry["Population"] = -1
                    else:                    
                        pop = re.search("[0-9,]*", pop.group(0))
                    
                        if pop == None:
                            entry["Population"] = -1
                        else:
                            entry["Population"] = pop.group(0)
                else:
                    entry["Population"] = pop.group(0)
                
                # Number of temples
                # Main Gods Worshipped
                Main_Gods = []
                num_temple = 0
                for templedata in re.finditer("\.*(main god|Main altar|dedicated)([^.]+)\.", entry["Raw_Text"]["Village_Temples"], flags=re.IGNORECASE):
                    # print(uid, templedata)
                    for god in godsList:
                        if god["Name"] in templedata.group(0):
                            if not god["Name"] in Main_Gods:
                                Main_Gods.append(god["Name"])
                        else:
                            for aliases in god["Aliases"]:
                                if aliases in templedata.group(0):
                                    if not god["Name"] in Main_Gods:
                                        Main_Gods.append(god["Name"])
                                        
                    num_temple = num_temple + 1
                
                entry["Num_Temples"] = str(num_temple)
                entry["Main_Gods"] = Main_Gods
                entry["Num_Main_Gods"] = str(len(Main_Gods))
                
                # Secondary Gods Worshipped
                Sec_Gods = []
                for templedata in re.finditer("\.*(secondary)([^.]+)\.", entry["Raw_Text"]["Village_Temples"], flags=re.IGNORECASE):
                    # print(uid, templedata)
                    for god in godsList:
                        if god["Name"] in templedata.group(0):
                            if not god["Name"] in Sec_Gods:
                                Sec_Gods.append(god["Name"])
                        else:
                            for aliases in god["Aliases"]:
                                if aliases in templedata.group(0):
                                    if not god["Name"] in Sec_Gods:
                                        Sec_Gods.append(god["Name"])
                
                entry["Secondary_Gods"] = Sec_Gods
                entry["Num_Secondary_Gods"] = str(len(Sec_Gods))
                
                villages.append(entry) 
                
    infile.close()
    
    # dump to file
    with open("./output/village_master.json", "w") as outfile:
        json.dump(villages, outfile, ensure_ascii=False)
        outfile.close()

    print("completed parsing", path)
    return villages
              

In [15]:
villages = []

overview = createVillageFiles("./input/1-100.txt", "Southern")
overview = createVillageFiles("./input/101-200.txt", "Southern")
overview = createVillageFiles("./input/201-297.txt", "Southern")
overview = createVillageFiles("./input/298-400.txt", "Northern")
overview = createVillageFiles("./input/401-500.txt", "Northern")
overview = createVillageFiles("./input/501-600.txt", "Northern")
overview = createVillageFiles("./input/601-659.txt", "Northern")
overview = createVillageFiles("./input/jiuliyang.txt", "Jiuliyang")
print("Completed")

completed parsing ./input/1-100.txt
completed parsing ./input/101-200.txt
completed parsing ./input/201-297.txt
completed parsing ./input/298-400.txt
completed parsing ./input/401-500.txt
completed parsing ./input/501-600.txt
completed parsing ./input/601-659.txt
completed parsing ./input/jiuliyang.txt
Completed


In [58]:
# Create csvs

import json, csv

with open('./output/village_master.json') as infile:
    
    indata = json.load(infile)
    
    # General data including 1) UID, 2) Name, 3) Name_zh, 4) Alliance, 5) Alliance_UID, 6) Alliance_zh, 7) Irrigation_Region
    keysToExtract = ["UID", "Name", "Name_zh", "Alliance", "Alliance_UID", "Alliance_zh", "Irrigation_Region"]
    genDataBuffer = []
        
    for entry in indata:
        
        outEntry = {}
        
        for var in entry:
            if var in keysToExtract:
                outEntry[var] = entry[var]
                
        genDataBuffer.append(outEntry)
        
    with open('./output/summary.csv', 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys, delimiter=',')
        writeheader(dict_writer)
        dict_writer.writerows(genDataBuffer)
        
    # Raw_Text
    
    

NameError: name 'writeheader' is not defined